# Running TCAV for Fire Engine Data

This is a notebook that shows an example on how to run the TCAV library to predict which concepts are most important in the prediction of the contents of an image. This example focuses on using colors (red, yellow, green and blue) as human interpretable concepts in the prediction of Fire Engine images using GoogleNet.

## Installing Packages
Make sure to have all the required packages before running by executing the following command:

In [ ]:
%pip install -r requirements.txt

## Downloading Data
The data used in the TCAV library is split into three categories: 
> **Targets:** The imagenet object category we are interested in (in this case that is fire engine)

> **Concepts:** The human comprehendable concepts used in the analysis

> **Random folders:** Random data obtained from imagenet

### Classes and Random Folders Data
To download the class data and the random folders data, start by downloading the [imagenet dataset](https://www.kaggle.com/competitions/imagenet-object-localization-challenge/data). For the Fire Engine example, the class that was used was `fire engine`. The random folders were also extraced from the same dataset by running the following command:

> python random_gen.py

ATH SKOÐA BETUR?

### Concept Data
To get the color concept data used in the Fire Engine example run the following commands (Make sure to change the source_dir argument):

> cd tcav/tcav_examples/image_models/imagenet

> python download_and_make_datasets.py --source_dir=YOUR_PATH --number_of_images_per_folder=50

## Importing Libraries


In [ ]:
#Automatically reload all changed code
%load_ext autoreload
%autoreload 2

In [ ]:
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib
import os 
import tensorflow as tf

## Step 1: Defining Parameters and Creating Local Folders

In the cell below a few variables are defined that will be used throughout the notebook. 

**activation_dir:** directory to store activations

**cav_dir:** directory to store CAVs

**bottlenecks:** the intermediate layers in your model included for TCAV.

**target, concepts:** the same as explained in the Downloading Data section above

In [ ]:
print ('REMEMBER TO UPDATE YOUR_PATH (where images, models are)!')
  
project_dir = 'tcav_dir'
# the name of the parent directory that results are stored (only if you want to cache)
project_name = 'tcav_class_test'
working_dir = "/tmp/" + project_dir + '/' + project_name
# where activations are stored (only if your act_gen_wrapper does so)
activation_dir =  working_dir + '/activations/'
# where CAVs are stored. 
# You can say None if you don't wish to store any.
cav_dir = working_dir + '/cavs/'
# where the images live.

# TODO: replace 'YOUR_PATH' with path to downloaded models and images. 
source_dir = 'YOUR_PATH'
bottlenecks = [ 'mixed3a', 'mixed3b','mixed4a', 'mixed4b','mixed4c','mixed4d','mixed4e','mixed5a','mixed5b', 'logit']  # @param 

utils.make_dir_if_not_exists(activation_dir)
utils.make_dir_if_not_exists(working_dir)
utils.make_dir_if_not_exists(cav_dir)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]

target = 'fire engine'  
concepts = ["red-c","yellow-c","blue-c","green-c"] 

## Step 2: Defining the Model

**GRAPH_PATH:** path to where the trained model is stored.

**LABEL_PATH:** path to where the labels are stored. Each line contains one class, and they are ordered with respect to their index in the logit layer.

In [ ]:
# Create TensorFlow session.
sess = utils.create_session()

# GRAPH_PATH is where the trained model is stored.
GRAPH_PATH = source_dir + "/inception5h/tensorflow_inception_graph.pb"

# LABEL_PATH is where the labels are stored. 
LABEL_PATH = source_dir + "/inception5h/imagenet_comp_graph_label_strings.txt"

#Creating the GoogleNet model
mymodel = model.GoogleNetWrapper_public(sess,
                                        GRAPH_PATH,
                                        LABEL_PATH)

# Step 3: 
SKRIFA MEIRA HÉR

In [ ]:
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=200)

# Step 4: Run TCAV

In [ ]:
import absl
absl.logging.set_verbosity(0)
num_random_exp=8

## only running num_random_exp = 10 to save some time. The paper number are reported for 500 random runs. 
for alpha in alphas:
    alpha = [alpha]
    mytcav = tcav.TCAV(sess,
                       target,
                       concepts,
                       bottlenecks,
                       act_generator,
                       alpha,
                       cav_dir=cav_dir,
                       num_random_exp=num_random_exp)#10)
    print ('Running for: ', alpha,'. This may take a while... Go get coffee!')
    results = mytcav.run(run_parallel=False)
    utils_plot.plot_results2(results, num_random_exp=num_random_exp, alpha=alpha)
    print ('done!')